# このコードの目的

支持率が伸びている/落ちているデータセットをサイズが同じになるように集積して、  
特徴分析を行う  
データセットが同じになるようにするとき、前処理した後のtxtファイルで比較するほうがよい
+  
機械学習にかける  

In [10]:
from 過去のコード.Mecab_func import *
import glob
import re
import sys
import os
import pandas as pd
import numpy as np

In [8]:
#各フォルダのサイズを比較するコード
#普通にエクスプローラーで見ればいい説



def compare_datasize(folder_A,folder_B):
    file_list_A=glob.glob(folder_A)
    file_list_B=glob.glob(folder_B)    
    A_size=[]
    B_size=[]
    
    for file_A in file_list_A:
        A_size.append(os.path.getsize(file_A))
        
    for file_B in file_list_B:
        B_size.append(os.path.getsize(file_B))    
        
    return(A_size,B_size)

compare_datasize(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\high_aproval_rate/*.txt',r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\low_aproval_rate/*.txt')

([484116, 87004, 70425, 312848, 402900, 233894, 249250, 94986, 404028, 35654],
 [279626, 109622, 82276, 102090, 60860, 109786, 247278, 148276, 99188, 153320])

In [11]:
def Preprocessing(fname):
    Matubi=[]
    with open(fname,errors='ignore') as data_file:
                ftitle, fext = os.path.splitext(fname)
                fname_out=ftitle+'Prep.txt'
                #空白文字を削除する
                text_data=data_file.read()
                text_data=re.sub('.+　','',text_data)
                #改行で区切る
                text_data=re.sub('\n','',text_data)
                #句点で区切り、最後の要素を抜き取る
                text_data=text_data.split('。')
                with open(fname_out, mode='w',errors='ignore') as out_file:
                    for i,text in enumerate(text_data):
                        #読点以前を削除する
    #                     text=text.split('、')
                        #なぜか先頭に空白文字が入るので取り除く↓。
                        text=text.lstrip()
                        #読点以降のみが集まる↓
                        text+='。'
                        out_file.write(text)
                    

# new_mecab (r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\2012-10-01.txt')    

for file in glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech2/*'):
    Preprocessing(file)

for file in glob.glob(r'C:/Users/icech/Desktop/share/Lab/2018_09_05/Docments/low_aproval_rate/*'):
    ftitle, fext = os.path.splitext(file)
    fname_out=ftitle+'.mecab'
    to_mecab(file,fname_out)
    
for file in glob.glob(r'C:/Users/icech/Desktop/share/Lab/2018_09_05/Docments/high_aproval_rate/*'):
    ftitle, fext = os.path.splitext(file)
    fname_out=ftitle+'.mecab'
    to_mecab(file,fname_out)    

In [15]:
def make_stopword():
    f = open("../docments/stopword.txt","r")
    list = []
    for x in f:
        list.append(x.rstrip("\n"))
        #以下のようにしてしまうと、改行コードがlistに入ってしまうため注意
        #list.append(x)
    f.close()
    return(list)



In [67]:
def new_count_morpheme(word_class):
    for i,file in enumerate(glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\low_aproval_rate\Mecab\*')):
        copus=[]
        print(i)
        stopword=make_stopword()
        #語数カウンター
        for line in make_lines(file):
            for morpheme in line:
                if len(morpheme['base'])==1 or morpheme['base'] in stopword:
                    continue
                if morpheme['pos'] == word_class:
                    word_counter.update([morpheme['base']])
                    #表層型をリスト型にする
                    copus.append(morpheme['base'])
                    copus.append(' ')
        copus=','.join(copus)
    return(copus,word_counter)

In [68]:
#品詞ごとに語数をカウントするコード
# concatを使うことによって異なる長さのdfを結合している

copus_dict={}
word_classes=['形容詞','動詞','名詞','接続詞']
df2=pd.DataFrame()
for word_class in word_classes:
    df1=pd.DataFrame()
    word_counter = Counter()
#         copus_dict[word_class+'word']= new_count_morpheme(word_class,file)[0]
    copus_dict[word_class]= new_count_morpheme(word_class)[1]
    count_word ,count_cnt =zip(*copus_dict[word_class].most_common())
    df1[word_class]=count_word
    df1[word_class+'_cnt']=count_cnt
    df2=pd.concat([df2,df1],axis=1)

        



0


c:\users\icech\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: generator 'make_lines' raised StopIteration
  import sys


1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [69]:
print(df2)
df2.to_csv('./df2.csv',encoding='shift_jis')

         形容詞  形容詞_cnt    動詞  動詞_cnt       名詞  名詞_cnt     接続詞  接続詞_cnt
0        厳しい    153.0   できる   916.0      *\n     860     そして    963.0
1         高い    135.0   進める   527.0       必要     569      また    747.0
2         強い     98.0   られる   374.0       政府     521     しかし    336.0
3        大きい     87.0    言う   315.0       国民     497    あるいは    326.0
4        新しい     75.0   つくる   310.0      お尋ね     434     例えば    235.0
5        やすい     57.0    守る   301.0       今後     393     そこで    204.0
6        幅広い     52.0   向ける   299.0       米国     366      及び    142.0
7         多い     48.0  取り組む   267.0       重要     363      かつ    125.0
8         早い     47.0    持つ   247.0       状況     339     つまり    111.0
9        力強い     44.0   いたす   245.0       世界     335     まずは     80.0
10        低い     39.0   含める   242.0       交渉     327      一方     77.0
11        よい     34.0    得る   236.0       指摘     324      ただ     71.0
12       難しい     32.0  踏まえる   219.0       確保     321      なお     51.0
13     ふさわしい     31.

PermissionError: [Errno 13] Permission denied: './df2.csv'

ここから下は使ってません！

In [50]:
verb_word ,verb_cnt =zip(*copus_dict['動詞'].most_common())

In [25]:

#mecabファイルを読み込んで語数を数えるコード
df1=pd.DataFrame()
for i,file in enumerate(glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\low_aproval_rate\Mecab\*')):
    month=re.sub( '.+Mecab.' ,'', file)
    month=re.sub('-01Prep.+', '' ,month)
    print(month)
    
    adjective_word,adjective_cnt=new_count_morpheme('形容詞',file)
    verb_word,verb_cnt=new_count_morpheme('動詞',file)
    noun_word,noun_cnt=new_count_morpheme('名詞',file)
    conjuction_word,conjuction_cnt=new_count_morpheme('接続詞',file)

    adjective_word ,adjective_cnt =zip(*adjective_cnt.most_common())
    try:
        verb_word ,verb_cnt =zip(*verb_cnt.most_common())
    except:
        pass   
    
    noun_word ,noun_cnt =zip(*noun_cnt.most_common())
    conjuction_word,conjuction_cnt=zip(*conjuction_cnt.most_common())
    
    
    adjective_data={'adj_cnt'.format(month):list(adjective_cnt),
                    'adj_word':list(adjective_word)}
    adjective_df=pd.DataFrame(adjective_data)

    
    verb_data={'verb_cnt':list(verb_cnt),
               'verb_word':list(verb_word)}        
    verb_df=pd.DataFrame(verb_data)

    noun_data={'noun_cnt':list(noun_cnt),
               'noun_word':list(noun_word)}
#                'space':list(np.zeros(len(noun_cnt)))}
    noun_df=pd.DataFrame(noun_data)
    
    conjuction_data={'conjuction_cnt':list(conjuction_cnt),
               'conjuction_word':list(conjuction_word)}        
    conjuction_df=pd.DataFrame(conjuction_data)
    
    
    df2=pd.concat([adjective_df,verb_df,noun_df,conjuction_df],axis=1)
    print(df2)
    if i==0:
        df1=df2.copy()
    else:
        df1=pd.concat([df1,df2])

    
    



2014-01


c:\users\icech\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: generator 'make_lines' raised StopIteration
  


      adj_cnt adj_word  verb_cnt verb_word  noun_cnt noun_word  \
0        36.0      厳しい     164.0       できる       257       お尋ね   
1        27.0       高い     150.0       進める       155        実現   
2        23.0      幅広い     146.0       向ける       136        政府   
3        22.0       強い     104.0      取り組む       124        今後   
4        17.0      やすい      88.0       つくる       113        国民   
5        17.0      美しい      87.0      踏まえる       111        制度   
6        15.0    ふさわしい      72.0       目指す       110        重要   
7        15.0      大きい      67.0        持つ       108        検討   
8        15.0      新しい      63.0        図る       103        必要   
9        12.0       広い      58.0        守る        98        世界   
10       12.0       早い      57.0       られる        96        経済   
11       11.0      力強い      55.0       いたす        91        改革   
12       11.0      ほしい      52.0       含める        88        支援   
13        7.0       多い      42.0        得る        87        対策   
14        

      adj_cnt adj_word  verb_cnt verb_word  noun_cnt noun_word  \
0         9.0       高い      64.0       できる        49        地方   
1         7.0      大きい      44.0       進める        46        地域   
2         7.0      厳しい      26.0       つくる        41        必要   
3         6.0      力強い      26.0      取り組む        39        事実   
4         6.0      やすい      20.0       いたす        33        政策   
5         5.0      親しい      19.0       られる        33       お尋ね   
6         3.0       よい      19.0        図る        27        国民   
7         3.0       良い      19.0        働く        26        状況   
8         3.0       多い      18.0        持つ        25        課題   
9         3.0       強い      17.0        見る        25        検討   
10        3.0      難しい      17.0       上がる        24        経済   
11        2.0      正しい      15.0        守る        23        指摘   
12        2.0      美しい      14.0        言う        23        政府   
13        2.0       低い      12.0       しまう        23        秘密   
14        

      adj_cnt adj_word  verb_cnt verb_word  noun_cnt noun_word  \
0        11.0      厳しい      29.0       できる        51        必要   
1         6.0       早い      26.0        守る        42        政府   
2         5.0      大きい      24.0       られる        41        要件   
3         3.0      やすい      21.0        得る        34        国民   
4         2.0       長い      16.0       変わる        30        平和   
5         2.0      幅広い      12.0       いたす        25       自衛隊   
6         2.0       深い      12.0        言う        25        行使   
7         2.0      難しい      11.0        持つ        21        指摘   
8         1.0       遅い      10.0        もつ        21    平和安全法制   
9         1.0  言うまでもない      10.0        取る        21        機雷   
10        1.0      間近い      10.0       禁じる        19        確保   
11        1.0      危うい       9.0      取り巻く        19        考え   
12        1.0       よい       9.0        至る        18      安全保障   
13        1.0       尊い       9.0       当たる        18        国会   
14        

      adj_cnt adj_word  verb_cnt verb_word  noun_cnt  noun_word  \
0        35.0       高い     183.0       できる       493        *\n   
1        35.0      厳しい     111.0       進める       147         米国   
2        26.0       強い      64.0       られる       124         世界   
3        21.0      新しい      61.0        守る       123        ルール   
4        18.0       多い      60.0        示す       113       自由貿易   
5        17.0     粘り強い      60.0        言う        97         協定   
6        16.0      大きい      53.0       向ける        92         交渉   
7        11.0      やすい      52.0        持つ        89         経済   
8        10.0      力強い      50.0       つくる        83         年金   
9        10.0       広い      48.0       いたす        82         自由   
10        9.0       早い      42.0      取り組む        68         必要   
11        9.0      正しい      42.0        得る        67        お尋ね   
12        8.0      幅広い      39.0       しまう        66         今後   
13        8.0      難しい      37.0       含める        66         会

      adj_cnt adj_word  verb_cnt verb_word  noun_cnt   noun_word  \
0        10.0       強い      62.0       できる        81          犯罪   
1         9.0      厳しい      27.0       られる        56          必要   
2         8.0       高い      23.0        含む        50          締結   
3         7.0       低い      20.0        言う        50      テロ等準備罪   
4         5.0  言うまでもない      19.0       含める        48          条約   
5         4.0       幼い      19.0      踏まえる        45          テロ   
6         3.0       よい      16.0       向ける        45          負担   
7         3.0      幅広い      16.0        限る        43         北朝鮮   
8         2.0     よろしい      16.0        おく        41          処罰   
9         2.0      正しい      15.0       求める        40          米国   
10        2.0      少ない      15.0        示す        38          対象   
11        2.0       多い      14.0       果たす        33          重要   
12        2.0      大きい      14.0       進める        33          政府   
13        2.0      難しい      13.0        至る      

In [26]:
df3.head()
print(df3)

      adj_cnt adj_word  verb_cnt verb_word  noun_cnt noun_word  \
0        36.0      厳しい     164.0       できる       257       お尋ね   
1        27.0       高い     150.0       進める       155        実現   
2        23.0      幅広い     146.0       向ける       136        政府   
3        22.0       強い     104.0      取り組む       124        今後   
4        17.0      やすい      88.0       つくる       113        国民   
5        17.0      美しい      87.0      踏まえる       111        制度   
6        15.0    ふさわしい      72.0       目指す       110        重要   
7        15.0      大きい      67.0        持つ       108        検討   
8        15.0      新しい      63.0        図る       103        必要   
9        12.0       広い      58.0        守る        98        世界   
10       12.0       早い      57.0       られる        96        経済   
11       11.0      力強い      55.0       いたす        91        改革   
12       11.0      ほしい      52.0       含める        88        支援   
13        7.0       多い      42.0        得る        87        対策   
14        

In [28]:
df3.to_csv('./cmp.csv')